### 데이터 확인 및 불러오기

- 데이터는 구글 드라이브 > 프로젝트 > 중급 프로젝트 > 원본데이터에 있어요.
- 제공된 문서 파일들과 `data_list.csv` 파일에 담긴 메타데이터를 확인합니다.
- 문서 파일을 불러옵니다. `hwp`와 `pdf` 두 가지 포맷에 대응할 수 있어야 해요.
- 유용한 메타데이터가 무엇일지 판단하여 함께 활용해 보세요.

### 문서 청킹

- 청크 크기, 그리고 청크 간 중첩 크기를 잘 설정하여 문서를 청킹합니다.
- (심화) 제안서 포맷을 활용해 의미 단위까지 고려하여 문서를 청킹해 보고 성능을 확인해 보세요.

### 임베딩 생성

- 임베딩 성능과 비용을 고려해 적절한 임베딩 모델을 선정합니다.
- 사용의 편의성과 검색 기능을 고려해 적절한 Vector DB를 선정합니다.

### Retrieval 기능 구현

- 먼저 naive한 retrieval을 구현해 베이스라인으로 삼으면 좋습니다. 베이스라인을 기반으로 점점 더 retrieval 기능을 고도화해 나가면 됩니다.
- 데이터 소스 문서가 하나가 아니라 여러 개이기 때문에, 타깃 문서를 정확히 찾기 위해 메타데이터 필터링을 활용할 수 있습니다. 이때 사용자가 발주 기관이나 사업명 같은 정보를 비슷하지만 정확하지는 않게 입력하는 케이스도 고려하는 게 좋아요.
- (심화) Retrieval과 관련해 다양한 옵션과 기법을 실험하면서 성능을 확인해 보세요.
    - Retrieval을 위한 프롬프트 엔지니어링
    - Top-k 검색에서 k 값 설정
    - 단순 유사도 기반 검색 / MMR(Maximum Marginal Relevance) / Hybrid Search
    - Multi-Query, Re-Ranking 등 심화 기법

### Generation 기능 구현

- 텍스트 생성 능력과 비용을 고려해 적절한 언어 모델을 선정합니다.
- 원하는 답변 양상과 분량에 따라서 temperature, top_p, max tokens 등 텍스트 생성과 관련된 옵션도 적절하게 설정해 주세요.
- 답변 생성을 위한 최적의 프롬프트를 작성합니다.
    - Retrieval 과정에서 찾은 컨텍스트를 충실히 반영해야 합니다.
    - 불필요한 내용이 답변에 포함되지 않도록 합니다.
    - 응답의 톤이나 스타일을 조정합니다.
    - 비용을 고려해 토큰 사용량도 최적화해 보세요.
- RAG 시스템이 대화 맥락을 유지하여 사용자와 대화를 이어 갈 수 있도록 해 주세요.

### 성능 평가

- 답변의 품질을 평가할 때는 다음과 같은 기준을 사용할 수 있습니다.
    - 사용자가 요청한 내용을 단일 문서에서 정확하게 뽑아내 답변하는지
    - 사용자가 여러 문서에 대해 요청한 내용을 잘 종합해서 답변하는지
    - 후속 질문의 맥락을 잘 이해하고 답변하는지
    - 문서에 포함되어 있지 않은 내용에 대해서는 모른다고 답변하는지
- 위와 같은 기준을 어떤 방식으로 평가하고 어떤 지표로 나타낼지 선정해 보세요.
- 주어진 문서 데이터에 맞게 다양한 질문 세트를 준비하여 성능 평가에 활용해 보세요.
    - 질문 예시
        - 국민연금공단이 발주한 이러닝시스템 관련 사업 요구사항을 정리해 줘.
            - 콘텐츠 개발 관리 요구 사항에 대해서 더 자세히 알려 줘.
            - 교육이나 학습 관련해서 다른 기관이 발주한 사업은 없나?
        - 기초과학연구원 극저온시스템 사업 요구에서 AI 기반 예측에 대한 요구사항이 있나?
            - 그럼 모니터링 업무에 대한 요청사항이 있는지 찾아보고 알려 줘.
        - 한국 원자력 연구원에서 선량 평가 시스템 고도화 사업을 발주했는데, 이 사업이 왜 추진되는지 목적을 알려 줘.
        - 고려대학교 차세대 포털 시스템 사업이랑 광주과학기술원의 학사 시스템 기능개선 사업을 비교해 줄래?
            - 고려대학교랑 광주과학기술원 각각 응답 시간에 대한 요구사항이 있나? 문서를 기반으로 정확하게 답변해 줘.
- 품질 좋은 답변도 중요하지만 응답 속도가 너무 느려져서도 안 됩니다.

In [ ]:
# Google Drive Mount
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Load data_list.csv and find NaN values

Load the `data_list.csv` file into a pandas DataFrame and check for missing (NaN) values.

In [ ]:
import pandas as pd
import os

data_dir = '/content/drive/MyDrive/Data'
csv_path = os.path.join(data_dir, 'data_list.csv')

try:
    # Load the CSV file into a pandas DataFrame
    data_list_df = pd.read_csv(csv_path)

    # Display the first few rows of the DataFrame
    print("First 5 rows of data_list.csv:")
    display(data_list_df.head())

    # Check for NaN values in each column
    print("\nChecking for NaN values:")
    nan_counts = data_list_df.isnull().sum()

    # Display the count of NaN values per column
    print("Number of NaN values per column:")
    print(nan_counts)

except FileNotFoundError:
    print(f"Error: The file {csv_path} was not found.")
except Exception as e:
    print(f"An error occurred while reading the CSV file or checking for NaN values: {e}")

First 5 rows of data_list.csv:


,공고 번호,공고 차수,사업명,사업 금액,발주 기관,공개 일자,입찰 참여 시작일,입찰 참여 마감일,사업 요약,파일형식,파일명,텍스트
0,20241001798,0.0,한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보시스템 고도화,130000000.0,한영대학,2024-10-04 13:51:23,NaN,2024-10-15 17:00:00,- 한영대학교 특성화 맞춤형 교육환경 구축을 위해 트랙운영 학사정보시스템을 고도화한...,hwp,한영대학_한영대학교 특성화 맞춤형 교육환경 구축 - 트랙운영 학사정보.hwp,\n \n2024년 특성화 맞춤형 교육환경 구축 – 트랙운영 학사정보시스템 ...
1,20241002912,0.0,2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선,129300000.0,한국연구재단,2024-10-04 15:01:52,2024-10-14 10:00:00,2024-10-16 14:00:00,- 사업 개요: 2024년 대학 산학협력활동 실태조사 시스템(UICC) 기능개선\n...,hwp,한국연구재단_2024년 대학산학협력활동 실태조사 시스템(UICC) 기능개선.hwp,\r\n \r\n \r\n \r\n제 안 요 청 서\r\n[ 2024년 대학 ...
2,20240827859,0.0,EIP3.0 고압가스 안전관리 시스템 구축 용역,40000000.0,한국생산기술연구원,2024-08-28 11:31:02,2024-08-29 09:00:00,2024-09-09 10:00:00,- 사업 개요: EIP3.0 고압가스 안전관리 시스템 구축 용역\n- 추진배경: 안...,hwp,한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.hwp,\r\n \r\nEIP3.0 고압가스 안전관리\r\n시스템 구축 용역\...
3,20240430918,0.0,도시계획위원회 통합관리시스템 구축용역,150000000.0,인천광역시,2024-04-18 16:26:32,2024-05-02 10:00:00,2024-05-09 16:00:00,- 사업명: 도시계획위원회 통합관리시스템 구축 용역\n- 용역개요: 도시계획위원회와...,hwp,인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp,\r\n \r\n \r\n도시계획위원회 통합관리시스템 구축\r\n제 안 요 청...
4,20240430896,0.0,봉화군 재난통합관리시스템 고도화 사업(협상)(긴급),900000000.0,경상북도 봉화군,2024-04-18 16:33:28,2024-04-26 09:00:00,2024-04-30 17:00:00,- 사업명: 봉화군 재난통합관리시스템 고도화 사업\n- 사업개요: 공동수급(공동이행...,hwp,경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).hwp,\r\n \r\n \r\n제안요청서\r\n \r\n사 업 명\r\n봉화...



Checking for NaN values:
Number of NaN values per column:
공고 번호        18
공고 차수        18
사업명           0
사업 금액         1
발주 기관         0
공개 일자         0
입찰 참여 시작일    26
입찰 참여 마감일     8
사업 요약         0
파일형식          0
파일명           0
텍스트           0
dtype: int64


In [ ]:
%pip install easyocr

In [ ]:
%pip install pyhwp hwp-extract

In [ ]:
%pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 101.0 MB/s eta 0:00:00


In [ ]:
import os

data_dir = '/content/drive/MyDrive/Data'
print(f"Listing contents of directory: {data_dir}")
try:
    for item in os.listdir(data_dir):
        print(item)
except FileNotFoundError:
    print(f"Directory not found: {data_dir}")
except Exception as e:
    print(f"An error occurred while listing directory contents: {e}")

Listing contents of directory: /content/drive/MyDrive/Data
files
data_list.csv
data_list.gsheet


In [ ]:
import os
import sys
import fitz  # PyMuPDF
import easyocr
import numpy as np
from PIL import Image
import subprocess

def find_executable(executable_name):
    """Finds the full path to an executable within the environment's PATH or script directories."""
    # Check in PATH first
    for path in os.environ["PATH"].split(os.pathsep):
        exe_path = os.path.join(path, executable_name)
        if os.path.isfile(exe_path) and os.access(exe_path, os.X_OK):
            return exe_path

    # Check in common script directories relative to sys.executable
    script_dirs = [
        os.path.join(os.path.dirname(sys.executable), 'bin'), # Linux/macOS
        os.path.join(os.path.dirname(sys.executable), 'Scripts') # Windows
    ]
    for script_dir in script_dirs:
        exe_path = os.path.join(script_dir, executable_name)
        if os.path.isfile(exe_path) and os.access(exe_path, os.X_OK):
            return exe_path

    return None


# Define HWP extraction function using pyhwp based on typical usage
def extract_text_from_hwp(hwp_path):
    """
    Extracts text from an HWP file using pyhwp's hwp5txt command.

    Args:
        hwp_path (str): The path to the HWP file.

    Returns:
        str: The extracted text from the HWP.
    """
    text = ""
    hwp5txt_path = find_executable('hwp5txt')

    if not hwp5txt_path:
        print("Error: hwp5txt command not found. Make sure pyhwp is installed correctly.")
        return ""

    try:
        # Use hwp5txt command-line tool via subprocess
        result = subprocess.run([hwp5txt_path, hwp_path], capture_output=True, text=True, encoding='utf-8')
        if result.returncode == 0:
            text = result.stdout
        else:
            print(f"Error extracting text from HWP {hwp_path} using hwp5txt: {result.stderr}")
            text = ""
    except Exception as e:
        print(f"Error processing HWP {hwp_path}: {e}")
        text = ""
    return text

# Define function for PDF text extraction using easyOCR
def extract_text_from_pdf_easyocr(pdf_path):
    """
    Extracts text from a PDF file using easyOCR.

    Args:
        pdf_path (str): The path to the PDF file.

    Returns:
        str: The extracted text from the PDF.
    """
    # Initialize easyocr reader for Korean and English (adjust languages as needed)
    reader = easyocr.Reader(['ko', 'en'])
    text = ""
    try:
        pdf_document = fitz.open(pdf_path)
        for page_num in range(pdf_document.page_count):
            page = pdf_document.load_page(page_num)

            # Render page to an image (as a numpy array) for easyOCR
            pixmap = page.get_pixmap()
            img = Image.frombytes("RGB", [pixmap.width, pixmap.height], pixmap.samples)
            img_np = np.array(img)

            # Use easyOCR to read text from the image
            result = reader.readtext(img_np, detail=0) # detail=0 returns only the text
            text += " ".join(result) + "\n" # Join lines with a space and add newline between pages

        pdf_document.close()
    except Exception as e:
        print(f"Error processing PDF {pdf_path} with easyOCR: {e}")
        text = "" # Return empty string in case of processing error

    return text


data_dir = '/content/drive/MyDrive/Data/files'
extracted_texts = {}

print(f"Attempting to process files in directory: {data_dir}")

if not os.path.isdir(data_dir):
    print(f"Error: Directory not found at {data_dir}")
else:
    files_in_dir = os.listdir(data_dir)
    if not files_in_dir:
        print(f"No files found in directory: {data_dir}")
    else:
        for filename in files_in_dir:
            file_path = os.path.join(data_dir, filename)
            if os.path.isfile(file_path):
                if filename.lower().endswith('.pdf'):
                    print(f"Processing PDF with easyOCR: {filename}...")
                    try:
                        text = extract_text_from_pdf_easyocr(file_path) # Use easyOCR function
                        extracted_texts[filename] = text
                        print(f"Successfully extracted text from {filename} using easyOCR")
                    except Exception as e:
                        print(f"Failed to process PDF {filename} with easyOCR: {e}")
                        extracted_texts[filename] = "" # Store empty string for failed files
                elif filename.lower().endswith('.hwp'):
                    print(f"Processing HWP with pyhwp: {filename}...")
                    try:
                        text = extract_text_from_hwp(file_path) # Use pyhwp function
                        extracted_texts[filename] = text
                        print(f"Successfully extracted text from {filename} using pyhwp")
                    except Exception as e:
                        print(f"Failed to process HWP {filename} with pyhwp: {e}")
                        extracted_texts[filename] = "" # Store empty string for failed files
                else:
                    print(f"Skipping unsupported file type: {filename}")


        print("\nExtraction complete.")
        print(f"Total files found in directory: {len(files_in_dir)}")
        print(f"Files processed (PDF/HWP): {len(extracted_texts)}")

        # Display the keys (filenames) of the extracted texts dictionary
        print("\nFilenames with extracted text:")
        for fname in extracted_texts.keys():
            print(fname)

Attempting to process files in directory: /content/drive/MyDrive/Data/files
Processing HWP with pyhwp: 대한장애인체육회_2025년 전국장애인체육대회 전산 및 시스템, 홈페이지 .hwp...
Successfully extracted text from 대한장애인체육회_2025년 전국장애인체육대회 전산 및 시스템, 홈페이지 .hwp using pyhwp
Processing HWP with pyhwp: 인천광역시_인천일자리플랫폼 정보시스템 구축 ISP 수립용역.hwp...
Successfully extracted text from 인천광역시_인천일자리플랫폼 정보시스템 구축 ISP 수립용역.hwp using pyhwp
Processing HWP with pyhwp: 국방과학연구소_기록관리시스템 통합 활용 및 보안 환경 구축.hwp...
Successfully extracted text from 국방과학연구소_기록관리시스템 통합 활용 및 보안 환경 구축.hwp using pyhwp
Processing HWP with pyhwp: 인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp...
Successfully extracted text from 인천광역시_도시계획위원회 통합관리시스템 구축용역.hwp using pyhwp
Proc

Successfully extracted text from 한국수자원공사_건설통합시스템(CMS) 고도화.hwp using pyhwp
Processing PDF with easyOCR: 서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf...
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% CompleteMuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

Successfully extracted text from 서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf using easyOCR
Processing HWP with pyhwp: 전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.hwp...
Successfully extracted text from 전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.hwp using pyhwp
Processing HWP with pyhwp: 한국전기안전공사_전기안전 관제시스템 보안 모듈 개발 용역.hwp...
Successfully extracted text from 한국전기안전공사_전기안전 관제시스템 보안 모듈 개발 용역.hwp using pyhwp
Processing HWP with pyhwp: 한국수자원공사_용인 첨단 시스템반도체 국가산단 용수공급사업 타당성.hwp...
Successfully extracted text from 한국수자원공사_용인 첨단 시스템반도체 국가산단 용수공급사업 타당성.hwp using pyhwp
Pr

KeyboardInterrupt: 